In [2]:
import surprise
import pandas as pd
import math 
from surprise import Dataset
from surprise import Reader
#import matplotlib.pyplot as plt
import numpy as np
from surprise import accuracy
#from sklearn.metrics import confusion_matrix
from surprise import KNNWithMeans, KNNBaseline, KNNBasic, SVDpp, SlopeOne, NMF, NormalPredictor, KNNWithZScore, BaselineOnly, CoClustering, SVD
#from surprise.model_SELECTion import cross_validate
import pyodbc
#!pip install pyodbc
#!pip install opencv-python
#!pip install matplotlib
import matplotlib as plt
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Connect to SQL Server

In [3]:


server = 'CamLaptop'  
database = 'Foo'  
conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establish a connection
conn = pyodbc.connect(conn_str) 

Pull training data set from SQL Server

In [4]:
# Create a cursor object
cursor = conn.cursor()

# Execute SQL queries
query = """ 
			SELECT ContID, viewed_resources.[user_id], viewed_resources.resource_id,rating, relevant, helpful, presentDifferently, CustID, title, [type],competency,subCompetency from user_feedback 

			INNER JOIN 

			viewed_resources ON user_feedback.[viewed_resource_id] = viewed_resources.id 

			INNER JOIN 

			(SELECT [user_id], COUNT(*) AS num_views, ROW_NUMBER() OVER (ORDER BY [user_id] ASC) AS CustID
			FROM viewed_resources GROUP BY [user_id]) AS A ON A.[user_id] = viewed_resources.[user_id]

			INNER JOIN 

			resources_content ON viewed_resources.resource_id = resources_content.id

			INNER JOIN 

			(SELECT resource_id, COUNT(*) AS num_views, ROW_NUMBER() OVER (ORDER BY [resource_id] ASC) AS ContID
			FROM viewed_resources GROUP BY resource_id) AS B ON B.resource_id = viewed_resources.resource_id

			WHERE EXISTS

			(SELECT count([user_id]) as cnt, [user_id] from user_feedback inner join viewed_resources 
			 ON user_feedback.[viewed_resource_id] = viewed_resources.id group by [user_id] having count([user_id]) > 5) 
    	"""

cursor.execute(query)


# take query and write output to a dataframe 
dfo = pd.DataFrame.from_records(cursor.fetchall(), columns=[column[0] for column in cursor.description])



Generate List of Unique Values for UserID, Type, Competency

In [ ]:
#Generate List of Unique Values for UserID, Type, Competency
type_List = dfo['type'].unique().tolist()
user_List = dfo['CustID'].unique().tolist()
competency_List = dfo['competency'].unique().tolist()
content_List = dfo['ContID'].unique().tolist()

print(competency_List)
print(type_List)
print(user_List)
print(content_List)

Create Training Data Sets For Three Unique Models

In [6]:
df_comp = dfo[['CustID','competency','rating']]
df_type = dfo[['CustID','type','rating']]
df_resource = dfo[['CustID','ContID','rating']]


Create Models and Train With User Data

In [ ]:
reader = Reader(rating_scale=(0, 1)) 
#Competency Model
data_comp = Dataset.load_from_df(df_comp[["CustID", "competency", "rating"]], reader)
trainset = data_comp.build_full_trainset()
sim_options = {
      "name": "cosine",
      "user_based": True,
      "k" : 20
  }
  # Create the algorithm
algo = KNNWithMeans(sim_options=sim_options)

  # Train the algorithm on the training set
algo.fit(trainset)


#Content Type Model
data_comp = Dataset.load_from_df(df_type[["CustID", "type", "rating"]], reader)
trainset = data_comp.build_full_trainset()
sim_options = {
      "name": "cosine",
      "user_based": True,
      "k" : 25
  }
  # Create the algorithm
algo1 = KNNWithMeans(sim_options=sim_options)

  # Train the algorithm on the training set
algo1.fit(trainset)

#Content Type Model
data_comp = Dataset.load_from_df(df_resource[["CustID", "ContID", "rating"]], reader)
trainset = data_comp.build_full_trainset()
sim_options = {
      "name": "cosine",
      "user_based": True,
      "k" : 20
  }
  # Create the algorithm
algo2 = KNNWithMeans(sim_options=sim_options)

  # Train the algorithm on the training set
algo2.fit(trainset)

Generate Predictions For Each Model, Content Tyoe, and User

In [8]:
type_List = dfo['type'].unique().tolist()
user_List = dfo['CustID'].unique().tolist()
competency_List = dfo['competency'].unique().tolist()
resource_List = dfo['ContID'].unique().tolist()

df_resource = dfo[['CustID','resource_id','rating']]

#Create Dataframe for predictions 
dfz = pd.DataFrame(columns=['UserID', 'Competency', 'Rating', 'ValueType'])
# Loop over users
for users in user_List:
    # Loop over competencies
    for comp in competency_List:
        # Predict the rating for the user and competency
        prediction = algo.predict(users, comp)
        # Extract the necessary information from the prediction object
        uid = prediction.uid
        iid = prediction.iid
        est = prediction.est
        # Append the prediction information to the dataframe
        dfz = dfz.append({'UserID': uid, 'Competency': iid, 'Rating': est, 'ValueType': 'Competency'}, ignore_index=True)

#Create Dataframe for predictions 
dfk = pd.DataFrame(columns=['UserID', 'Type', 'Rating', 'ValueType'])
for users in user_List:
    # Loop over content types
    for types in type_List:
        # Predict the rating for the user and competency
        prediction = algo1.predict(users, types)     
        # Extract the necessary information from the prediction object
        uid = prediction.uid
        iid = prediction.iid
        est = prediction.est     
        # Append the prediction information to the dataframe
        dfk = dfk.append({'UserID': uid, 'Type': iid, 'Rating': est, 'ValueType': "Type"}, ignore_index=True)  

#Create Dataframe for predictions 
dfc = pd.DataFrame(columns=['UserID', 'ContentID', 'Rating', 'ValueType'])
for users in user_List:
    # Loop over content types
    for types in resource_List:
        # Predict the rating for the user and competency
        prediction = algo2.predict(users, types)     
        # Extract the necessary information from the prediction object
        uid = prediction.uid
        iid = prediction.iid
        est = prediction.est     
        # Append the prediction information to the dataframe
        dfc = dfc.append({'UserID': uid, 'ContentID': iid, 'Rating': est, 'ValueType': "ContentID"}, ignore_index=True)  

Format and Union Predictions For SQL Injestion 

In [10]:
DFK = dfk.sort_values(by=['UserID','Rating'], ascending=[True,False])
DFZ = dfz.sort_values(by=['UserID','Rating'], ascending=[True,False])
DFZ['Type'] = DFZ['Competency']
DFC = dfc.sort_values(by=['UserID','Rating'], ascending=[True,False])
DFC['Type'] = DFC['ContentID']

In [11]:
DFX = pd.concat([DFK, DFZ, DFC])
DFX =DFX.drop(['ContentID', 'Competency'], axis = 1)

Load Predictions Into SQL Server's Prediction Table

In [12]:
conn = pyodbc.connect(conn_str) 
cursor = conn.cursor()
query = """ Delete [user_predictions] """
cursor.execute(query)

for index, row in DFX.iterrows():
    insert_query = f"INSERT INTO {'user_predictions'} (UserID, ContID, Rating, ValueType) VALUES (?, ?, ?, ?)"
    cursor.execute(insert_query, row['UserID'], row['Type'], row['Rating'], row['ValueType'])

#cursor.execute(query)
conn.commit()


Query Prediction Table to Grab Data For Recomendation 

In [13]:
conn = pyodbc.connect(conn_str) 
cursor = conn.cursor()
query = """ 

select UserID, user_predictions.ContID as ContID, Rating, ValueType from user_predictions LEFT OUTER JOIN (SELECT ContID, CustID from user_feedback 

			INNER JOIN 

			viewed_resources ON user_feedback.[viewed_resource_id] = viewed_resources.id 

			INNER JOIN 

			(SELECT [user_id], COUNT(*) AS num_views, ROW_NUMBER() OVER (ORDER BY [user_id] ASC) AS CustID
			FROM viewed_resources GROUP BY [user_id]) AS A ON A.[user_id] = viewed_resources.[user_id]

			INNER JOIN 

			resources_content ON viewed_resources.resource_id = resources_content.id

			INNER JOIN 

			(SELECT resource_id, COUNT(*) AS num_views, ROW_NUMBER() OVER (ORDER BY [resource_id] ASC) AS ContID
			FROM viewed_resources GROUP BY resource_id) AS B ON B.resource_id = viewed_resources.resource_id

			WHERE EXISTS

			(SELECT count([user_id]) as cnt, [user_id] from user_feedback inner join viewed_resources 
			 ON user_feedback.[viewed_resource_id] = viewed_resources.id group by [user_id] having count([user_id]) > 5)) AS A ON CAST(A.ContID AS INT) = UserID AND user_predictions.ContID = CAST(A.ContID AS INT) WHERE ValueType = 'ContentID' AND A.ContID is null
			UNION 
			select * from user_predictions where user_predictions.ValueType <> 'ContentID' ORDER BY UserID,ValueType,Rating   desc
             
               """

cursor.execute(query)

# take query and write output to a dataframe 
dfPred1 = pd.DataFrame.from_records(cursor.fetchall(), columns=[column[0] for column in cursor.description])

In [14]:
conn = pyodbc.connect(conn_str) 
cursor = conn.cursor()
query = """ 

select * from user_predictions
             
               """

cursor.execute(query)

# take query and write output to a dataframe 
dfPred = pd.DataFrame.from_records(cursor.fetchall(), columns=[column[0] for column in cursor.description])

In [15]:
dfPred_Type = dfPred[dfPred['ValueType'] == 'Type']
dfPred_Cont = dfPred1[dfPred1['ValueType'] == 'ContentID']
dfPred_Comp = dfPred[dfPred['ValueType'] == 'Competency']

Grab User ID and Generate Predictions

In [ ]:
#1,2,5

user = int(input())

c = dfPred_Comp[dfPred_Comp['UserID'] == user].iloc[0]
t = dfPred_Type[dfPred_Type['UserID'] == user].iloc[0]
id = dfPred_Cont[dfPred_Cont['UserID'] == user].iloc[0]
# Load the image
print('Competency:',c.ContID,'\nType:',t.ContID )
if c.ContID == 'SelfAwareness' and t.ContID == 'ASK':
    
    image_path = r'C:\Project\CompRec\Images\62192751414d65002e768309.png'
    image = cv2.imread(image_path)


    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()
else:
    image_path = r'C:\Project\CompRec\Images\63695586c9b2193989ea3ab0.png'
    image = cv2.imread(image_path)

  
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

print('ContentID#:',id.ContID)
if id.ContID == '1':
    image_path = r'C:\Project\CompRec\Images\63193c6899924a3177032b37.png'
    image = cv2.imread(image_path)


    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

elif id.ContID == '85':

    image_path = r'C:\Project\CompRec\Images\631ce0eb99924a3177037cd0.png'
    image = cv2.imread(image_path)


    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

else:
    image_path = r'C:\Project\CompRec\Images\63dd6d220966a99f43a1060a.png'
    image = cv2.imread(image_path)


    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

Close SQL Connections For Good Measure

In [ ]:
cursor.close()
conn.close()